In [1]:
import yaml
with open("../data/raw/bold_api/bold_names.yaml", "r") as f:
    config = yaml.safe_load(f)
config["bold_name"]

['Agapetus',
 'Ameletus inopinatus',
 'Amphinemura borealis',
 'Baetis rhodani',
 'Baetis vernus',
 'Capnopsis schilleri',
 'Diura',
 'Elmis aenea',
 'Ephemerella aurivillii',
 'Ephemerella mucronata',
 'Heptagenia sulphurea',
 'Hydraena',
 'Hydropsyche pellucidula',
 'Hydropsyche saxonica',
 'Hydropsyche siltalai',
 'Isoperla',
 'Kageronia fuscogrisea',
 'Lepidostoma hirtum',
 'Leptophlebia',
 'Leuctra nigra',
 'Leuctra',
 'Limnius volckmari',
 'Micrasema gelidum',
 'Micrasema setiferum',
 'Nemoura cinerea',
 'Nemoura',
 'Neureclipsis bimaculata',
 'Oulimnius tuberculatus',
 'Oxyethira',
 'Plectrocnemia',
 'Polycentropus flavomaculatus',
 'Polycentropus irroratus',
 'Protonemura',
 'Rhyacophila nubila',
 'Sialis',
 'Silo pallipes',
 'Simuliidae',
 'Sphaerium',
 'Taeniopteryx nebulosa']

In [2]:
from urllib.request import urlretrieve
from pathlib import Path

raw_folder = Path("../data/raw/bold_api")
out_folder = Path("../data/processed/bold_api")

def fetch_bold_sequences(taxon, out_fname):
    url = ("http://v3.boldsystems.org/index.php/API_Public/sequence?"
    f"taxon={taxon.replace(' ', '%20')}"
        "&marker=COI-5P"
        "&format=xml")

    urlretrieve(url, filename=out_fname)
    print(f"loaded '{taxon}'")

taxon = "Sialis"
out_fname = raw_folder / f"{taxon.replace(' ', '_')}.fasta"
fetch_bold_sequences(taxon, out_fname)

loaded 'Sialis'


In [3]:
for taxon in config["bold_name"]:
    out_fname = raw_folder / f"{taxon.replace(' ', '_')}.fasta"
    fetch_bold_sequences(taxon, out_fname)

loaded 'Agapetus'
loaded 'Ameletus inopinatus'
loaded 'Amphinemura borealis'
loaded 'Baetis rhodani'
loaded 'Baetis vernus'
loaded 'Capnopsis schilleri'
loaded 'Diura'
loaded 'Elmis aenea'
loaded 'Ephemerella aurivillii'
loaded 'Ephemerella mucronata'
loaded 'Heptagenia sulphurea'
loaded 'Hydraena'


KeyboardInterrupt: 

In [4]:
from Bio import SeqIO
from random import sample
from io import StringIO
import pandas as pd
import numpy as np

def preprocess_fasta(input):
    folder = input.parent
    records = [r for r in SeqIO.parse(input, "fasta")]
    print("Original length:", len(records))

    # Find the lengths of sequences and filter too long and short sequences
    lengths = np.asarray([len(r.seq) for r in records])
    filter_ind = np.logical_and((lengths <= 658), (lengths >= 600))
    records_subset0 = [records[i] for i in np.where(filter_ind)[0]]
    print("Sequences between 600 and 658:", len(records_subset0))

    # Remove duplicate sequences
    seen = set()
    records_subset = []
    for r in records_subset0:
        if r.seq not in seen:
            seen.add(r.seq)
            records_subset.append(r)

    print("After duplicates removed:", len(records_subset))

    # Cap the maximum number of sequences to 500
    if len(records_subset) > 200:
        records_subset = sample(records_subset, 200)
        print("Capped to 200:", len(records_subset))

    subset_name = raw_folder / f"{input.stem}_subset.fasta"
    with open(subset_name, "w") as output_handle:
        SeqIO.write(records_subset, output_handle, "fasta")
    print(subset_name)


In [5]:
import subprocess
for taxon in config["bold_name"]:
    out_fname = raw_folder / f"{taxon.replace(' ', '_')}.fasta"
    preprocess_fasta(out_fname)

Original length: 652
Sequences between 600 and 658: 596
After duplicates removed: 359
Capped to 200: 200
..\data\raw\bold_api\Agapetus_subset.fasta
Original length: 42
Sequences between 600 and 658: 41
After duplicates removed: 23
..\data\raw\bold_api\Ameletus_inopinatus_subset.fasta
Original length: 14
Sequences between 600 and 658: 14
After duplicates removed: 7
..\data\raw\bold_api\Amphinemura_borealis_subset.fasta
Original length: 588
Sequences between 600 and 658: 413
After duplicates removed: 341
Capped to 200: 200
..\data\raw\bold_api\Baetis_rhodani_subset.fasta
Original length: 159
Sequences between 600 and 658: 102
After duplicates removed: 55
..\data\raw\bold_api\Baetis_vernus_subset.fasta
Original length: 13
Sequences between 600 and 658: 10
After duplicates removed: 9
..\data\raw\bold_api\Capnopsis_schilleri_subset.fasta
Original length: 74
Sequences between 600 and 658: 57
After duplicates removed: 37
..\data\raw\bold_api\Diura_subset.fasta
Original length: 331
Sequences b

In [6]:
def read_records(filename):
    return [r for r in SeqIO.parse(filename, "fasta")]

In [7]:
for taxon in config["bold_name"]:
    subset_name = raw_folder / f"{taxon.replace(' ', '_')}_subset.fasta"
    align_name = out_folder / f"{subset_name.stem}.afa"
    records = read_records(subset_name)
    print(f"Aligning {taxon}, {len(records)} records")
    subprocess.run([r"..\muscle.exe",
                        "-super5", str(subset_name), 
                        "-output", str(align_name)]
                        )
    print(f"Aligned to file {align_name}")


Aligning Agapetus, 200 records
Aligned to file ..\data\processed\bold_api\Agapetus_subset.afa
Aligning Ameletus inopinatus, 23 records
Aligned to file ..\data\processed\bold_api\Ameletus_inopinatus_subset.afa
Aligning Amphinemura borealis, 7 records
Aligned to file ..\data\processed\bold_api\Amphinemura_borealis_subset.afa
Aligning Baetis rhodani, 200 records
Aligned to file ..\data\processed\bold_api\Baetis_rhodani_subset.afa
Aligning Baetis vernus, 55 records
Aligned to file ..\data\processed\bold_api\Baetis_vernus_subset.afa
Aligning Capnopsis schilleri, 9 records
Aligned to file ..\data\processed\bold_api\Capnopsis_schilleri_subset.afa
Aligning Diura, 37 records
Aligned to file ..\data\processed\bold_api\Diura_subset.afa
Aligning Elmis aenea, 73 records
Aligned to file ..\data\processed\bold_api\Elmis_aenea_subset.afa
Aligning Ephemerella aurivillii, 29 records
Aligned to file ..\data\processed\bold_api\Ephemerella_aurivillii_subset.afa
Aligning Ephemerella mucronata, 17 records
Al

In [11]:
records = []
for taxon in config["bold_name"]:
    subset_name = raw_folder / f"{taxon.replace(' ', '_')}_subset.fasta"
    align_name = out_folder / f"{subset_name.stem}.afa"
    for r in read_records(align_name):
        records.append(r)

with open("../data/processed/fb2.fasta", "w") as output_handle:
    SeqIO.write(records, output_handle, "fasta")